In [1]:
import tensorflow as tf
# tf.__version__

In [2]:
import zipfile,os
 
base_dir = '../input/4-animal-classification/'
train_dir = os.path.join(base_dir, 'train')

In [3]:
os.listdir('../input/4-animal-classification/train')

['horse', 'dog', 'cat', 'deer']

# Preprocessing

In [47]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
image_datagen = ImageDataGenerator(
#                     rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
                    validation_split = 0.2,
                    width_shift_range=0.1,
                    height_shift_range=0.1,)

# Rescale kalo resolusi gambar seragam
# Ga perlu rescale kalo resolusi beda-beda

train_generator = image_datagen.flow_from_directory(
        train_dir,
        subset='training',
        target_size=(384, 384),
        batch_size=32,
        class_mode='categorical'
        )

validation_generator = image_datagen.flow_from_directory(
        train_dir,
        subset='validation',
        target_size=(384, 384),
        batch_size=32,
        class_mode='categorical'
        )

Found 2240 images belonging to 4 classes.
Found 560 images belonging to 4 classes.


In [ ]:
model = tf.keras.models.Sequential([
    # Input
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Blok 1
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Blok 2
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Blok 3
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Blok 3
    tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.GlobalAveragePooling2D(),

    #Flatten
    tf.keras.layers.Flatten(),
    
    # Fully
    tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    
    # 4 classes
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

# Transfer Learning

In [51]:
from tensorflow.keras.layers import Input , Dense , Flatten , GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from efficientnet_v2 import EfficientNetV2S

transfer = EfficientNetV2S(
              input_shape=(384,384,3),
              include_top=False,
              weights='imagenet'
)

# Include Top
# Kalo True -> Fully Connected Layer bawaan (Output Layer)
# Kalo False -> Fully Connected Layer bisa di modifikasi (Output Layer)

model = tf.keras.models.Sequential([
    transfer,
    tf.keras.layers.GlobalAveragePooling2D(),

    #Flatten
    tf.keras.layers.Flatten(),
    
    # Fully
    tf.keras.layers.Dense(640, activation='relu'),
#     tf.keras.layers.Dense(512, activation='relu'),
    # tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(32, activation='relu'),
    # tf.keras.layers.Dense(16, activation='relu'),
#     tf.keras.layers.Dense(8, activation='relu'),
    
    # 4 classes
    tf.keras.layers.Dense(4, activation='softmax')
])

model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetv2-s (Functional (None, 12, 12, 1280)      20331360  
_________________________________________________________________
global_average_pooling2d_17  (None, 1280)              0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 1280)              0         
_________________________________________________________________
dense_51 (Dense)             (None, 640)               819840    
_________________________________________________________________
dense_52 (Dense)             (None, 128)               82048     
_________________________________________________________________
dense_53 (Dense)             (None, 4)                 516       
Total params: 21,233,764
Trainable params: 21,079,892
Non-trainable params: 153,872
___________________________________

In [54]:
# compile model dengan 'SGD' optimizer loss function 'categorical_crossentropy' 
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9), 
            loss='categorical_crossentropy',
            metrics = ['accuracy'])

In [55]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

class stopTraining(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy') > 0.99 and logs.get('val_accuracy') < 100):
      print('Val Akurasi telah melebihi 99%')
      self.model.stop_training = True

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
checkpoint = ModelCheckpoint("checkpoints/best.h5", monitor='val_accuracy', verbose=1, save_best_only=True)
callBack = stopTraining()

In [56]:
# latih model dengan model.fit 
history = model.fit(
      train_generator,                    
      epochs=20,                              # tambahkan epochs jika akurasi model belum optimal
      validation_data=validation_generator,   # menampilkan akurasi pengujian data validasi
      callbacks = [early, checkpoint, callBack],
      verbose=2
)

Epoch 1/20


2022-09-21 09:45:33.514110: W tensorflow/core/common_runtime/bfc_allocator.cc:457] Allocator (GPU_0_bfc) ran out of memory trying to allocate 27.00MiB (rounded to 28311552)requested by op sequential_17/efficientnetv2-s/block6-14_expand_conv/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-09-21 09:45:33.514643: I tensorflow/core/common_runtime/bfc_allocator.cc:1004] BFCAllocator dump for GPU_0_bfc
2022-09-21 09:45:33.514687: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (256): 	Total Chunks: 1021, Chunks in use: 1001. 255.2KiB allocated for chunks. 250.2KiB in use in bin. 108.5KiB client-requested in use in bin.
2022-09-21 09:45:33.514703: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (512): 	Total Chunks: 553, Chunks in use: 536. 357.2KiB allocated for chunks. 348.2KiB in use in bin. 310.7Ki

ResourceExhaustedError:  OOM when allocating tensor with shape[32,1536,12,12] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_17/efficientnetv2-s/block6-14_expand_conv/Conv2D (defined at tmp/ipykernel_18/2498471402.py:7) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_326192]

Function call stack:
train_function


In [32]:
results = model.evaluate(validation_generator)
print(f"test loss {results[0]}, test accuracy {results[1]}")

18/18 [==============================] - 22s 1s/step - loss: 0.0521 - accuracy: 0.9839
test loss 0.05213194340467453, test accuracy 0.9839285612106323


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(16, 5))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_path = '../input/4-animal-classification/test/test'
test_gen = image_datagen.flow_from_directory(    
    directory=test_path,
    target_size=(300, 300),
    batch_size=32,
    classes=['.'],
    shuffle=False,
)

In [ ]:
from PIL import Image
predict = []

In [ ]:
import numpy as np
for i in range(1,730):
    img = Image.open('../input/4-animal-classification/test/test/'+str(i)+'.jpg')
    img = img.resize((300,300))
    c_predict = model.predict(np.asarray(img).reshape(-1,300,300,3))
    predict.append(c_predict)
    if(i % 100 == 0):
        print(i, "Predicted")

In [ ]:
import pandas as pd
predictions = np.argmax(predict,axis=2)
submission_file = pd.read_csv("../input/4-animal-classification/Sample_submission.csv")
submission_file['Label'] = predictions
submission_file.to_csv('submission3.csv', index=False)
submission_file['Label'].value_counts()

In [ ]:
submission_file